In [3]:
import numpy as np 

import scipy.linalg as sla

import math

import matplotlib.pyplot as plt

%matplotlib inline

$\textbf{Практическая задача 1}$

$\dfrac{\partial}{\partial x}\left(u \dfrac{\partial u}{\partial x}\right) + \dfrac{\partial}{\partial y}\left(u \dfrac{\partial u}{\partial y}\right) = - \dfrac{5\pi^2}{8}\sin(\pi x)\left(\sin(\frac{\pi y}{2} + \cos(\frac{\pi y}{2})\right)$, $x \in (0, 1)$, $y \in (0, 1)$

$u(0, y) = u(1, y) = 0$

$u(x, 0) = u(x, 1) = \sqrt{\sin(\pi x)}$

$\textbf{1. Построение разностной задачи}$

Аппроксимируем производные по схеме "крест". Узловые точки: $v_{m,n}$, где $n, m \in \{0, 1, 2, ..., N\}$, $N = \frac{1}{h}$, $h$ - шаг сетки, одинаковый по обоим измерениям. 

$\dfrac{\partial}{\partial x}\left(u \dfrac{\partial u}{\partial x}\right) \approx \dfrac{1}{h}\left(u_{m+0.5,n} \cdot \dfrac{u_{m+1,n} - u_{m,n}}{h} - u_{m-0.5,n} \cdot \dfrac{u_{m,n} - u_{m-1,n}}{h} \right)$

$\dfrac{\partial}{\partial y}\left(u \dfrac{\partial u}{\partial y}\right) \approx \dfrac{1}{h}\left(u_{m,n+0.5} \cdot \dfrac{u_{m,n+1} - u_{m,n}}{h} - u_{m,n-0.5} \cdot \dfrac{u_{m,n} - u_{m,n-1}}{h} \right)$

Точки в половинных ущлах аппроксимируем средним арифметическим значений в ближайших узлах. Например, $u_{m+0.5,n} = \dfrac{u_{m,n} + u_{m+1,n}}{2}$. Тогда для производных получим:

$\dfrac{\partial}{\partial x}\left(u \dfrac{\partial u}{\partial x}\right) \approx \dfrac{1}{2h^2}\left((u_{m+1,n})^2 + (u_{m-1,n})^2 - 2(u_{m,n})^2 \right)$

$\dfrac{\partial}{\partial y}\left(u \dfrac{\partial u}{\partial y}\right) \approx \dfrac{1}{2h^2}\left((u_{m,n+1})^2 + (u_{m,n-1})^2 - 2(u_{m,n})^2 \right)$

Разностная задача: 
    
$ \dfrac{1}{2h^2}\left((u_{m+1,n})^2 + (u_{m-1,n})^2 - 2(u_{m,n})^2 \right) + \dfrac{1}{2h^2}\left((u_{m,n+1})^2 + (u_{m,n-1})^2 - 2(u_{m,n})^2 \right) = -\dfrac{5\pi^2}{8}\sin(\pi x_m)\left(\sin(\frac{\pi y_n}{2} + \cos(\frac{\pi y_n}{2})\right)$

Здесь $x_m = hm$, $y_n = hn$.

$u_{m,0} = u_{m,N} = 0$, $u_{0,n} = u_{N,n} = \sqrt{\sin(\pi x_m)}$ для любых допустимых значений $m$ и $n$.

$\textbf{2. Метод установки}$

Вместо стационарной рассмотрим нестационарную задачу: 

$\dfrac{\partial u}{\partial t} = \dfrac{\partial}{\partial x}\left(u \dfrac{\partial u}{\partial x}\right) + \dfrac{\partial}{\partial y}\left(u \dfrac{\partial u}{\partial y}\right) + \dfrac{5\pi^2}{8}\sin(\pi x_m)\left(\sin(\frac{\pi y_n}{2} + \cos(\frac{\pi y_n}{2})\right)$

Ее решение при достаточно большом $t$ сходится к решению нестационарной задачи. 

Решим нестационарную задачу $\textbf{методом простых итераций}$. Для этого аппроксимируем производную по времени: 

$\dfrac{\partial u}{\partial t} = \dfrac{u^{s+1} - u^s}{\tau}$

Здесь $s$ - узел сетки по времени и одновременно порядок итерации, $\tau$ - временной шаг.

Таким образом, нестационарная разностная задача:
    
$\dfrac{u^{s+1}_{m,n} - u^s_{m,n}}{\tau} = \dfrac{1}{2h^2}\left((u^s_{m+1,n})^2 + (u^s_{m-1,n})^2 - 2(u^s_{m,n})^2 \right) + \dfrac{1}{2h^2}\left((u^s_{m,n+1})^2 + (u^s_{m,n-1})^2 - 2(u^s_{m,n})^2 \right) + \dfrac{5\pi^2}{8}\sin(\pi x_m)\left(\sin(\frac{\pi y_n}{2} + \cos(\frac{\pi y_n}{2})\right)$

$u_{m,0} = u_{m,N} = 0$, $u_{0,n} = u_{N,n} = \sqrt{\sin(\pi x_m)}$ для любых допустимых значений $m$ и $n$.

Выражение для "нового" значения в узле $(m,n)$:

$u^{s+1}_{m,n} = u^s_{m,n} + \dfrac{5\pi^2}{8}\sin(\pi x_m)\left(\sin(\frac{\pi y_n}{2} + \cos(\frac{\pi y_n}{2})\right) + \dfrac{\tau}{2h^2}\left((u^s_{m-1,n})^2 + (u^s_{m+1,n})^2 + (u^s_{m,n-1})^2 + (u^s_{m,n+1})^2 - 4(u^s_{m,n})^2 \right)$

$\textbf{3. Реализация задачи}$

Шаг сетки выберем равным $h = 0.2$. 

In [4]:
h = 0.2

Вычислим $\lambda_{min}$ и $\lambda_{max}$ - минимальное и максимальное собственные значения оператора Лапласа. В дальнейшем они будут использованы для выбора $\tau$ и количества итераций метода:

$\lambda_{min} = 2\pi^2$, $\lambda_{max} = \dfrac{8}{h^2} - 2*\pi^2$

In [5]:
lambda_min = 2 * math.pi * math.pi
print("lambda_min = {}\n".format(lambda_min))

lambda_min = 19.739208802178716



In [6]:
lambda_max = (8 / h ** 2.) - 2 * math.pi
print("lambda_max = {}\n".format(lambda_max))

lambda_max = 193.71681469282038



Метод простых итераций сходится, если выполнено условие: $\tau < \dfrac{2}{\lambda_{max}}$.

In [7]:
tau_max = 2 / lambda_max
print("tau < {}\n".format(tau_max))

tau < 0.01032434898938138



Выберем $\tau = \dfrac{2}{\lambda_{min} + \lambda_{max}}$.

In [8]:
tau = 2 / (lambda_min + lambda_max)
print("tau = {}\n".format(tau))

tau = 0.009369611441519505



Количество итераций $S$, необходимых для обеспечения сходимости с заданной точностью $\epsilon = 0.1$, равно: 

$S = \dfrac{\ln\epsilon}{\ln\rho}$, где $\rho = \dfrac{1 - \xi}{1 + \xi}$, $\xi = \dfrac{\lambda_{min}}{\lambda_{max}}$

In [9]:
epsilon = 0.01

In [10]:
xi = lambda_min / lambda_max
print("xi = {}\n".format(xi))

xi = 0.10189724022398093



In [11]:
rho = (1 - xi) / (1 + xi)
print("rho = {}\n".format(rho))

rho = 0.8150512833605638



In [12]:
S = math.log(epsilon) / math.log(rho)
print("S = {}\n".format(S))

S = 22.518702353077174



Установим количество итераций $total > S$.

In [13]:
total = int(S) + 1
print("total = {}\n".format(total))

total = 23



Программный код, реализующий итерации:

In [14]:
# подсчет правой части в точках m, n

def get_right_part(m, n):
    ans = (-5 * math.pi ** 2 / 8) * math.sin(math.pi * h * m) * (math.sin(math.pi * h * n / 2) + math.cos(math. pi * h * n / 2))
    return ans

In [15]:
print(get_right_part(1, 1))

-4.568717763101125


In [16]:
# подсчет нового значения в узле (m, n)

def get_new_value(prev_values, m, n):
    combination = (prev_values[m - 1][n]) ** 2 + (prev_values[m + 1][n]) ** 2 + (prev_values[m][n - 1]) ** 2 + (prev_values[m][n + 1]) ** 2
    ans = prev_values[m][n] - tau * get_right_part(m, n) + (tau / (2 * h * h)) * (combination - 4 * (prev_values[m][n]) ** 2)
    
    return ans

In [17]:
# реализация итераций
# начальные значения во всех узлах нулевые

size = int(1 / h + 1.)
prev_values = np.zeros((size, size))
cur_values = np.zeros((size, size))

for row in range(size):
    prev_values[row][0] = math.sqrt(math.sin(math.pi * h * row))
    prev_values[row][size - 1] = math.sqrt(math.sin(math.pi * h * row))

for it in range(total):
    for row in range(1, size - 1):
        for col in range(1, size - 1):
            cur_values[row][col] = get_new_value(prev_values, row, col)
    
    prev_values = cur_values
    for row in range(size):
        prev_values[row][0] = math.sqrt(math.sin(math.pi * h * row))
        prev_values[row][size - 1] = math.sqrt(math.sin(math.pi * h * row))
    
print(cur_values)

[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [7.66671541e-01 8.65442253e-01 9.13089388e-01 9.13425944e-01
  8.66138971e-01 7.66671541e-01]
 [9.75221265e-01 1.10149929e+00 1.16222620e+00 1.16251416e+00
  1.10209288e+00 9.75221265e-01]
 [9.75221265e-01 1.10173803e+00 1.16251358e+00 1.16274842e+00
  1.10222230e+00 9.75221265e-01]
 [7.66671541e-01 8.66137793e-01 9.13922948e-01 9.14105560e-01
  8.66516392e-01 7.66671541e-01]
 [1.10663761e-08 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 1.10663761e-08]]


$\textbf{4. Аналитическое решение задачи}$

Сделаем замену $v = u^2/2$, при которой исходное уравнение переходит в уравнение Пуассона с той же правой частью.

Будем искать решение в виде $v(x, y) = A\sin(\pi x)\left(\sin(\pi y/2) + \cos(\pi y/2)\right)$, $A = const$. Граничные условия удовлетворяются. Подстановка в исходное уравнение дает: 

$-A(\pi)^2\cdot\left(\sin(\pi y/2) + \cos(\pi y/2)\right) - A(\pi/2)^2\cdot\sin(\pi x)\cdot\ \left(\sin(\pi y/2) + \cos(\pi y/2)\right) = -\dfrac{5\pi^2}{8}\sin(\pi x)\left(\sin(\pi y/2) + \cos(\pi y/2)\right) $

$-A(1 + 0.25) = -0.625$

$A = 0.5$

Таким образом, $v = 0.5\sin(\pi x)\left(\sin(\pi y/2) + \cos(\pi y/2)\right)$; $u = \sqrt{\sin(\pi x)\left(\sin(\pi y/2) + \cos(\pi y/2)\right)}$.

$\textbf{5. Сравнение численного и аналитического решений}$

Вычислим истинные значения в узлах сеточной функции.

In [19]:
true_values = np.zeros((size, size))

for row in range(size):
    for col in range(size):
        true_values[row][col] = math.sqrt(math.sin(math.pi * row * h) * (math.sin(math.pi * col * h / 2) + math.cos(math.pi * col * h / 2)))
        
print(true_values)

[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [7.66671541e-01 8.60611774e-01 9.06101408e-01 9.06101408e-01
  8.60611774e-01 7.66671541e-01]
 [9.75221265e-01 1.09471509e+00 1.15257879e+00 1.15257879e+00
  1.09471509e+00 9.75221265e-01]
 [9.75221265e-01 1.09471509e+00 1.15257879e+00 1.15257879e+00
  1.09471509e+00 9.75221265e-01]
 [7.66671541e-01 8.60611774e-01 9.06101408e-01 9.06101408e-01
  8.60611774e-01 7.66671541e-01]
 [1.10663761e-08 1.24223387e-08 1.30789503e-08 1.30789503e-08
  1.24223387e-08 1.10663761e-08]]


Сравним полученные значения, вычитая истинные значения из приближенных:

In [22]:
np.set_printoptions(precision=4)
for col in true_values - cur_values:
    print(col)

[0. 0. 0. 0. 0. 0.]
[ 0.     -0.0048 -0.007  -0.0073 -0.0055  0.    ]
[ 0.     -0.0068 -0.0096 -0.0099 -0.0074  0.    ]
[ 0.     -0.007  -0.0099 -0.0102 -0.0075  0.    ]
[ 0.     -0.0055 -0.0078 -0.008  -0.0059  0.    ]
[0.0000e+00 1.2422e-08 1.3079e-08 1.3079e-08 1.2422e-08 0.0000e+00]


Видно, что аппроксимация прошла успешно.

In [23]:
# реализация итераций
# начальные значения во всех узлах нулевые

size = int(1 / h + 1.)
prev_values = np.zeros((size, size))
cur_values = np.zeros((size, size))

for row in range(size):
    prev_values[row][0] = math.sqrt(math.sin(math.pi * h * row))
    prev_values[row][size - 1] = math.sqrt(math.sin(math.pi * h * row))

In [24]:
prev_values

array([[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [7.6667e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        7.6667e-01],
       [9.7522e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        9.7522e-01],
       [9.7522e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        9.7522e-01],
       [7.6667e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        7.6667e-01],
       [1.1066e-08, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        1.1066e-08]])

In [382]:
# подсчет правой части в точках m, n

def get_right_part(m, n):
    ans =  (- 5 * math.pi ** 2 / 9) * (math.sin(math.pi * h * m / 3) + math.cos(math.pi * h * m / 3)) * math.sin(math.pi * h * n)
    return ans

In [340]:
def get_new_value(prev_values, m, n):
    combination = (prev_values[m - 1][n]) ** 2 + (prev_values[m + 1][n]) ** 2 + (prev_values[m][n - 1]) ** 2 + (prev_values[m][n + 1]) ** 2
    ans = prev_values[m][n] - tau * get_right_part(m, n) + (tau / (2 * h * h)) * (combination - 4 * (prev_values[m][n]) ** 2)
    
    return ans

In [27]:
#реализация итераций
# начальные значения во всех узлах нулевые

size = int(1 / h + 1.)
prev_values = np.zeros((size, size))
cur_values = np.zeros((size, size))

for row in range(size):
#     prev_values[row][size - 1] = math.sqrt(math.cos(math.pi * h * row / 2))
    
    prev_values[0][row] = math.sqrt(math.sin(math.pi * h * row))
    prev_values[size - 1][row] = math.sqrt((1 + math.sqrt(3)) * 0.5 * math.sin(math.pi * h * row / 2))

for it in range(total):
    for row in range(1, size - 1):
        for col in range(1, size - 1):
            cur_values[row][col] = get_new_value(prev_values, row, col)
    
    prev_values = cur_values
    for row in range(size):
        prev_values[row][size - 1] = math.sqrt(math.cos(math.pi * h * row / 2))
    
        prev_values[0][row] = math.sqrt(math.sin(math.pi * h * row / 2))
        prev_values[size - 1][row] = math.sqrt(0.5 * math.sin(math.pi * h * row / 2))
    
for x in cur_values:
    print(x)

[0.     0.5559 0.7667 0.8995 0.9752 1.    ]
[0.     0.7986 1.0221 1.1109 1.0971 0.9752]
[0.     0.9151 1.1477 1.2148 1.1461 0.8995]
[0.     0.8988 1.1215 1.1763 1.084  0.7667]
[0.     0.7406 0.9321 0.9856 0.9078 0.5559]
[0.     0.3931 0.5421 0.636  0.6896 0.7071]


In [355]:
true_values = np.zeros((size, size))

for row in range(size):
    for col in range(size):
        true_values[row][col] = math.sqrt((math.sin(math.pi * h * row / 3) + math.cos(math.pi * h * row / 3)) * math.sin(math.pi * h * col))
        
for x in true_values:
    print(x)

[0.0000e+00 7.6667e-01 9.7522e-01 9.7522e-01 7.6667e-01 1.1066e-08]
[0.0000e+00 8.3495e-01 1.0621e+00 1.0621e+00 8.3495e-01 1.2052e-08]
[0.0000e+00 8.8093e-01 1.1206e+00 1.1206e+00 8.8093e-01 1.2716e-08]
[0.0000e+00 9.0610e-01 1.1526e+00 1.1526e+00 9.0610e-01 1.3079e-08]
[0.0000e+00 9.1111e-01 1.1589e+00 1.1589e+00 9.1111e-01 1.3151e-08]
[0.0000e+00 8.9606e-01 1.1398e+00 1.1398e+00 8.9606e-01 1.2934e-08]


In [354]:
cur_values - true_values

array([[ 0.    , -0.2108, -0.2085, -0.0758,  0.2085,  1.    ],
       [ 0.    , -0.4344, -0.453 , -0.325 ,  0.0169,  0.9752],
       [ 0.    , -0.6006, -0.6694, -0.5703, -0.1769,  0.8995],
       [ 0.    , -0.6274, -0.7147, -0.6468, -0.2986,  0.7667],
       [ 0.    , -0.5628, -0.6364, -0.567 , -0.3045,  0.5559],
       [ 0.    , -0.503 , -0.5977, -0.5038, -0.2065,  0.7071]])

In [ ]:
# подсчет правой части в точках m, n

def get_right_part(m, n):
    ans =  (- 5 * math.pi ** 2 / 9) * (math.sin(math.pi * h * m / 3) + math.cos(math.pi * h * m / 3)) * math.sin(math.pi * h * n)
    return ans

def get_new_value(prev_values, m, n):
    combination = (prev_values[m - 1][n]) ** 2 + (prev_values[m + 1][n]) ** 2 + (prev_values[m][n - 1]) ** 2 + (prev_values[m][n + 1]) ** 2
    ans = prev_values[m][n] - tau * get_right_part(m, n) + (tau / (2 * h * h)) * (combination - 4 * (prev_values[m][n]) ** 2)
    
    return ans

#реализация итераций
# начальные значения во всех узлах нулевые

size = int(1 / h + 1.)
prev_values = np.zeros((size, size))
cur_values = np.zeros((size, size))

for row in range(size):
#     prev_values[row][size - 1] = math.sqrt(math.cos(math.pi * h * row / 2))
    
    prev_values[0][row] = math.sqrt(math.sin(math.pi * h * row))
    prev_values[size - 1][row] = math.sqrt((1 + math.sqrt(3)) * 0.5 * math.sin(math.pi * h * row / 2))

for it in range(total):
    for row in range(1, size - 1):
        for col in range(1, size - 1):
            cur_values[row][col] = get_new_value(prev_values, row, col)
    
    prev_values = cur_values
    for row in range(size):
        prev_values[row][size - 1] = math.sqrt(math.cos(math.pi * h * row / 2))
    
        prev_values[0][row] = math.sqrt(math.sin(math.pi * h * row / 2))
        prev_values[size - 1][row] = math.sqrt(0.5 * math.sin(math.pi * h * row / 2))
    
for x in cur_values:
    print(x)

true_values = np.zeros((size, size))

for row in range(size):
    for col in range(size):
        true_values[row][col] = math.sqrt((math.sin(math.pi * h * row / 3) + math.cos(math.pi * h * row / 3)) * math.sin(math.pi * h * col))
        
for x in true_values:
    print(x)

cur_values - true_values

In [36]:
cur_values

array([[0.    , 0.5559, 0.7667, 0.8995, 0.9752, 1.    ],
       [0.    , 0.7986, 1.0221, 1.1109, 1.0971, 0.9752],
       [0.    , 0.9151, 1.1477, 1.2148, 1.1461, 0.8995],
       [0.    , 0.8988, 1.1215, 1.1763, 1.084 , 0.7667],
       [0.    , 0.7406, 0.9321, 0.9856, 0.9078, 0.5559],
       [0.    , 0.3931, 0.5421, 0.636 , 0.6896, 0.7071]])

In [54]:
def f():
    global t
    t = 1.0031456134 * cur_values

In [55]:
t = np.zeros((size, size))

In [56]:
f()

In [57]:
t

array([[0.    , 0.5576, 0.7691, 0.9023, 0.9783, 1.0031],
       [0.    , 0.8012, 1.0253, 1.1144, 1.1006, 0.9783],
       [0.    , 0.918 , 1.1513, 1.2186, 1.1497, 0.9023],
       [0.    , 0.9016, 1.1251, 1.18  , 1.0874, 0.7691],
       [0.    , 0.7429, 0.9351, 0.9887, 0.9106, 0.5576],
       [0.    , 0.3943, 0.5438, 0.638 , 0.6918, 0.7093]])